In [10]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings
from collections import Counter
from torch.utils.data import Dataset
import torch
from pympler import asizeof
import torch.optim as optim 



import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
#from ExtractGenre import *
from CNN_ExtractGenre import *
from PolyphonicPreprocessing import *
import Util as Util

import DatasetLoader as DL
import Model as M



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [13]:
Path = os.path.realpath('clean_midi/Eminem/Stan.mid')
mid = mido.MidiFile(Path)
#print(mid)



Preprocessing: 100%|██████████| 15/15 [00:01<00:00, 12.44it/s]


In [88]:
torch.save(PolyDataset, 'PolyphonicDataset.pt')

# 1 Instrument 

In [2]:
Dataset = PreProcessing(nDir = 10)

Preprocessing: 100%|██████████| 10/10 [00:01<00:00,  5.15it/s]


In [89]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU available')
else:
    device = torch.device("cpu")
    print('GPU not available')


def LoadModel(Cond1D_Size, instrumentSize, Which):

    Path = os.path.realpath('ModelParameters')

    files = [
        ['generator_parameters.torch', 'gen_opt_state.torch'],
        ['Polygenerator_parameters2.torch', 'Polygen_opt_state2.torch']
    ]

    generator = M.Generator(input_size=100, cond_1d_size=Cond1D_Size, instrument_size=instrumentSize, n_hlayers=128)
    generator.apply(M.weights_init)
    generator.to(device)
    # Load the state dict previously saved
    generator_state_dict = torch.load(os.path.join(Path, files[Which][0]), map_location=torch.device('cpu'))
    # Update the network parameters
    generator.load_state_dict(generator_state_dict)

    gen_opt = optim.Adam(generator.parameters(), lr=2e-4)
    # Load the state dict previously saved
    gen_opt_state_dict = torch.load(os.path.join(Path, files[Which][1]), map_location=torch.device('cpu'))
    # Update the network parameters
    gen_opt.load_state_dict(gen_opt_state_dict)

    return generator

                                                    #MOnophonic or polyphonic
generator = LoadModel(Cond1D_Size=5, instrumentSize=4, Which=1)

GPU not available


In [90]:
PolyDataset = PolyphonicPreProcessing(nDir = 150)

Preprocessing: 100%|██████████| 150/150 [00:30<00:00,  4.98it/s]


In [ ]:
#np.shape(torch.tensor(np.random.randint(0, 2, (128, 16))).float().to(device).unsqueeze(0))





torch.Size([4, 128, 16])

In [112]:
generator.eval()
noise = torch.randn([1, 100], device=device)

Genre = 'rock'
bar = 51

prev_bar = PolyDataset[Genre][bar]['Bars'][0].to_dense().float().to(device)
InstrumentCode = PolyDataset[Genre][bar]['Program'][0]
Tempo = PolyDataset[Genre][bar]['Tempo'][0]

#prev_bar = torch.tensor(np.random.randint(0, 1, (4, 128, 16))).float().to(device)

cond_1d = torch.tensor([[Tempo] + InstrumentCode], dtype= torch.float32)
Bars = []
Bars.append(prev_bar)
#If polyphonic only 1 unsqueeze
prev_bar = prev_bar.unsqueeze(0)#.unsqueeze(0) 


for i in range(8):
   with torch.no_grad():
      generated_bar = generator(noise, prev_bar, cond_1d, 1)

   generated_bar=generated_bar.cpu().numpy()
   binary_bar = (generated_bar.reshape(4, 128, 16) > 0.3).astype(int)

   #if i % 2 == 0:
   #prev_bar = torch.tensor(binary_bar, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)
      #prev_bar = PolyDataset[Genre][bar+i]['Bars'][0].to_dense().float().to(device).unsqueeze(0)#.unsqueeze(0) 
   #prev_bar = torch.tensor(binary_bar, dtype=torch.float32, device=device).unsqueeze(0)


   Bars.append(binary_bar)



#ConcBars = np.concatenate(Bars, axis = 1)
PolyConcBars = np.concatenate(Bars, axis = 2)

#MonoBarsToMIDI(ConcBars, title='test', Instrument=InstrumentCode)

PolyBarsToMIDI(PolyConcBars, title='Polytest', Instrument=InstrumentCode)

print(InstrumentCode)

[33, 41, 49, 73]


In [94]:
polconc = []
for i in range(50, 54):
   for j in range(2):
      polconc.append(PolyDataset['rock'][i]['Bars'][j].to_dense())

conc = np.concatenate(polconc, axis = 2)
PolyBarsToMIDI(conc, title = 'True', Instrument = [33, 41, 49, 73])